<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

## 导入相关包，创建相机实例

In [1]:
from jetbot import Camera
from jetbot import bgr8_to_jpeg
import PID
camera = Camera.instance(width=720, height=720)

## 创建相关控制变量

In [2]:
global color_x, color_y, color_radius
color_x = color_y = color_radius = 0
global target_valuex
target_valuex = 2100
global target_valuey
target_valuey = 2048

## 创建存储HSV色域颜色分类数据的数组

In [3]:
import numpy as np
global color_lower
color_lower=np.array([156,43,46])
global color_upperv
color_upper = np.array([180, 255, 255])

## 创建PID控制实例

In [4]:
xservo_pid = PID.PositionalPID(1.9, 0.3, 0.35)
yservo_pid = PID.PositionalPID(1.5, 0.2, 0.3)

## 创建云台总线舵机实例

In [5]:
from servoserial import ServoSerial
servo_device = ServoSerial() 

SerialException: [Errno 13] could not open port /dev/ttyTHS1: [Errno 13] Permission denied: '/dev/ttyTHS1'

## 创建显示控件

In [ ]:
import cv2
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
color_image = widgets.Image(format='jpeg', width=300, height=300)
display(color_image)

### 颜色识别默认数组数据

In [12]:
color_lower=np.array([156,43,46])
color_upper = np.array([180, 255, 255])
target_valuex = target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

NameError: name 'servo_device' is not defined

### 设置识别红色数组数据

In [8]:
color_lower=np.array([0,43,46])
color_upper = np.array([10, 255, 255])
target_valuex = target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

### 设置识别黄色数组数据

In [ ]:
color_lower=np.array([26,43,46])
color_upper = np.array([34, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

### 设置识别蓝色数组数据

In [10]:
color_lower=np.array([100,43,46])
color_upper = np.array([124, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

### 设置识别绿色数组数据

In [11]:
color_lower=np.array([35,43,46])
color_upper = np.array([77, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

### 设置识别橙色数组数据

In [12]:
color_lower=np.array([11,43,46])
color_upper = np.array([25, 255, 255])
target_valuex = 2100
target_valuey = 2048
servo_device.Servo_serial_double_control(1, 2048, 2, 2048)

## 云台运动主进程

In [13]:
while 1:
    frame = camera.value
    frame = cv2.resize(frame, (300, 300))
    frame_=cv2.GaussianBlur(frame,(5,5),0)                    
    hsv=cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    mask=cv2.inRange(hsv,color_lower,color_upper)  
    mask=cv2.erode(mask,None,iterations=2)
    mask=cv2.dilate(mask,None,iterations=2)
    mask=cv2.GaussianBlur(mask,(3,3),0)     
    cnts=cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2] 
    if len(cnts)>0:
        cnt = max (cnts,key=cv2.contourArea)
        (color_x,color_y),color_radius=cv2.minEnclosingCircle(cnt)
        if color_radius > 10:
            # 将检测到的颜色标记出来
            cv2.circle(frame,(int(color_x),int(color_y)),int(color_radius),(255,0,255),2)  
            #Proportion-Integration-Differentiation
            xservo_pid.SystemOutput = color_x
            xservo_pid.SetStepSignal(150)
            xservo_pid.SetInertiaTime(0.01, 0.006)
            target_valuex = int(2100+xservo_pid.SystemOutput)
            # 输入Y轴方向参数PID控制输入
            yservo_pid.SystemOutput = color_y
            yservo_pid.SetStepSignal(150)
            yservo_pid.SetInertiaTime(0.01, 0.006)
            target_valuey = int(2048+yservo_pid.SystemOutput)
             # 将云台转动至PID调校位置
            servo_device.Servo_serial_double_control( 1, target_valuex, 2, target_valuey)
     # 实时传回图像数据进行显示
    color_image.value = bgr8_to_jpeg(frame)

KeyboardInterrupt: 